In [ ]:
import pandas as pd
import openai 
from keybert import KeyBERT
import langdetect
import time
import numpy as np

from transformers import pipeline

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 200)


# model = KeyBERT('distilbert-base-nli-mean-tokens')

In [ ]:
with open("apikey_moritz.txt", "r") as f:
    api_key = f.read()

print(api_key)
openai.api_key = api_key

## Text: Finetuned GPT3 Model


In [ ]:
fine_tune_list = openai.FineTune.list()
fine_tuned_model = fine_tune_list['data'][0].fine_tuned_model
print(fine_tuned_model)

In [ ]:
new_prompt = "A Google Maps review about a steak house. ###"
answer = openai.Completion.create(
  model=fine_tuned_model,
  prompt=new_prompt,
  max_tokens=200, # Change amount of tokens for longer completion
  temperature=1,
  stop = "END"
)
print(answer['choices'][0]['text'])

In [ ]:
def gpt3_finetuned(row):
    catgeoryName = row['categoryName'].lower()
    prompt = "A Google Maps review about a " + catgeoryName + ". ###"
    print(prompt)
    answer = openai.Completion.create(
      model=fine_tuned_model,
      prompt=prompt,
      max_tokens=200, # Change amount of tokens for longer completion
      temperature=1, 
      stop = "END"
    )
    return answer['choices'][0]['text']

# def gpt3_5_turbo(row):
#     prompt = "Write a Google Maps review about a " + row['categoryName'] + ". In Style of a Google user that writes a review about a place."
#     role = "In stlye of a google user that writes a review about a place."
#     prompt = "Write a Google Maps review about a " + row['categoryName']+"."
#     print(prompt)
#     return chatGPT(api_key, role).question(prompt)

#### Outdated: Nur für Test:  gpt3_5_turbo vs gpt3_finetuned 

In [ ]:
# ## df_test.to_csv('test_df_showcase20gpt.csv')
# # print("Done with gpt3_finetuned_v1 text generation.")

# df_test = pd.read_csv('test_df_showcase.csv')
# df_test['gpt3_finetuned_v1'] = df_test.apply(gpt3_finetuned, axis=1)
# df_test.to_csv('test_df_showcase.csv')
# print("Done with gpt3.5_turbo text generation.")

# print(df_test)
# print(df_test.shape)

In [ ]:
# df_test = pd.read_csv('test_df_showcase.csv')
# df_test['gpt3.5_turbo'] = df_test['gpt3.5_turbo'].astype(str)

# for index, row in df_test.iterrows():
#     print(row['title'])
#     print(row['categoryName'])
#     print("ORG:" + row['original_text'])
#     print("GPT:" + row['gpt3_finetuned_v1'])
#     print("GPT3.5:" + row['gpt3.5_turbo'])
#     print("__________________________________________________________________________________________")

### Jetzt tatsächliche Daten augmentiert / generiert. Dazu einen neuen Dataframe erstellen...

Später wird der dann bemerged.

In [ ]:
# df_true = pd.read_csv('base_keywords_sentiment_reduced.csv')
# df_finetune = pd.read_csv('dataset_ft_v3_en_mulitmod_pc.csv.csv')

# # print(df_true['categoryName'].value_counts())
# # print(df_finetune['categoryName'].value_counts())
# print(f'True:' , df_true.shape)

# df_fake = pd.DataFrame(columns=df_true.columns)
# df_fake['categoryName'] = df_true['categoryName']
# df_fake['label'] = 'FAKE'

# # just for reference. Can be delted later on. 
# df_fake['reviewId'] = df_true['reviewId']
# df_fake['title'] = df_true['title']
# df_fake['text'] = df_true['text']
# df_fake['placeId'] = df_true['placeId']

# df_fake.to_csv('fake_base.csv', index=False)

#### Ab hier neue finetunes ausprobieren und unter neuer Version speichern. 
 Wichtig! Spaltennamen ändern von gpt3_v3

In [ ]:
df_fake = pd.read_csv('fake_base_gpt3_v2331.csv')
df_fake["prompt_v4"] = "A Google Maps review about a " + df_fake["categoryName"] + ". ###"

for index, row in df_fake.iterrows():
    df_fake.at[index, 'gpt3_v4'] = gpt3_finetuned(row)
    print(df_fake.at[index, 'gpt3_v4'])
    print(index)
    if index % 50 == 0:
        df_fake.to_csv('fake_base_gpt3_v23314.csv', index=False)
        print("Wir warten ein paar Sekunden und hoffen, dass der API key deshalb nicht geblockt wird...")
        time.sleep(25)
        print("Weiter geht's...")

print("Done with gpt3 text generation.")
print(df_fake.shape)
print(df_fake.columns)

df_fake.to_csv("fake_base_gpt3_v23314.csv", index=False)

In [ ]:
# load df_fake and save in xlsx

df_fake = pd.read_csv('fake_base_gpt3_v23314.csv')
print(df_fake.shape)


# Image Generierung


Vorbereitung. Eine Sentiment-Analyse auf dem generierten Text durchführen. 

In [ ]:
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

In [ ]:
# do a sentiment analysis on the generated text gpt3_v2, gpt3_v3 gpt3_v3.1

df_fake = pd.read_csv('fake_base_gpt3_v23314.csv')
df_fake['sent_v2'] = np.nan
df_fake['sent_v3']= np.nan
df_fake['sent_v3.1'] = np.nan
df_fake['sent_v4'] = np.nan

df_fake['sent_v2'] = df_fake['gpt3_v2'].astype(str)
df_fake['sent_v3'] = df_fake['gpt3_v3'].astype(str)
df_fake['sent_v3.1'] = df_fake['gpt3_v3.1'].astype(str)
df_fake['sent_v4'] = df_fake['gpt3_v4'].astype(str)

for index, row in df_fake.iterrows():
    print("Processing row: ", index)
    try:
        df_fake.at[index, 'sent_v2'] = sentiment_task(row['gpt3_v2'])[0]['label']
        df_fake.at[index, 'sent_v3'] = sentiment_task(row['gpt3_v3'])[0]['label']
        df_fake.at[index, 'sent_v3.1'] = sentiment_task(row['gpt3_v3.1'])[0]['label']
        df_fake.at[index, 'sent_v4'] = sentiment_task(row['gpt3_v4'])[0]['label']
    except RuntimeError:
        print("Skipped due to messy data. Row: ", index)
        pass

df_fake.to_csv('fake_base_gpt3_v23314_sent.csv', index=False)
df_fake.to_excel('fake_base_gpt3_v23314_sent.xlsx', index=False)


## Image: Dall-E-2


In [ ]:
with open("apikey_mama.txt", "r") as f:
    openai.api_key = f.read()

In [ ]:
df_fake = pd.read_csv('fake_base_gpt3_v23314_sent.csv')
df_real = pd.read_csv('base_keywords_sentiment_reduced.csv')

print(df_real.shape)
print(df_fake.shape)

df_fake['reviewImageUrls/0'] = df_real['reviewImageUrls/0']
df_fake['reviewImageUrls/1'] = df_real['reviewImageUrls/1']
df_fake['reviewImageUrls/2'] = df_real['reviewImageUrls/2']
df_fake['reviewImageUrls/3'] = df_real['reviewImageUrls/3']
df_fake['reviewUrl'] = df_real['reviewUrl']

df_fake['fake_reviewImageUrls/0'] = np.nan
df_fake['fake_reviewImageUrls/1'] = np.nan
df_fake['fake_reviewImageUrls/2'] = np.nan
df_fake['fake_reviewImageUrls/3'] = np.nan

print(df_fake.columns)
print(df_fake.shape)

print("How many values in the column 'reviewImageUrls/0' are not null?")
print(df_fake['reviewImageUrls/0'].isna().sum())
print(df_fake['reviewImageUrls/1'].isna().sum())
print(df_fake['reviewImageUrls/2'].isna().sum())
print(df_fake['reviewImageUrls/3'].isna().sum())

print("How many na values are in the column 'fake_reviewImageUrls/'?")
print(df_fake['fake_reviewImageUrls/0'].isna().sum())
print(df_fake['fake_reviewImageUrls/1'].isna().sum())
print(df_fake['fake_reviewImageUrls/2'].isna().sum())
print(df_fake['fake_reviewImageUrls/3'].isna().sum())

df_fake.to_csv('fake_base_gpt3_v23314_sent.csv', index=False)

In [ ]:
def generate_dalle_prompt(row):
    label = row['label']
    categoryName = row['categoryName'].lower()
    sentiment = row['sent_v4'].lower()
    locationString = f"a {categoryName}"

    if sentiment == "positive":
        locationString = f"a great  stay at a {categoryName}"
    elif sentiment == "negative":
        locationString = f"a great {categoryName}"
    else:
        locationString = f"an average {categoryName}"

    PROMPT = f"A photograph of very nice experience at {locationString}, wide shot, wide-angle lens, hard focus, iPhone 13 Pro Max, lovly athmosphere, 4k, high resolution. "
    print(PROMPT)
    return PROMPT

prompt  = generate_dalle_prompt(df_fake.iloc[703])
print(df_fake.iloc[300]['gpt3_v4'])
response = openai.Image.create(prompt=prompt,n=1, size="256x256")
print(response["data"][0]["url"])

Download von Bildern 

In [ ]:
import requests
from PIL import Image
from io import BytesIO

response = requests.get("https://lh5.googleusercontent.com/p/AF1QipNf-ZejrPdXHfis5MCtO6kxNxIwa4s5MwyOxxwj=w256-h256-p-k-no")
img = Image.open(BytesIO(response.content))
img = img.convert("RGBA")
print(img.mode)
print(img.size)
img.save('test.png')

Anderer Ansatz, variationen von bestehenenden, echten Bildern

In [ ]:
df_fake = pd.read_csv('fake_base_gpt3_v23314.csv')
df_fake["prompt_dalle"] = ''
df_fake["dalle_url"] = ''
df_fake["dalle_url_2"] = ''

for index, row in df_fake.iterrows():
    catgeoryName = row['categoryName'].lower()
    PROMPT = f"A high quality photo of a lovely stay at a " + catgeoryName + ",wide shot, Sigma 85mm f/1.4 "
    print (index)
    print(PROMPT)
    response_url = "Hallo"
    # response = openai.Image.create(  prompt=PROMPT,n=1,  size="256x256")
    # print(response["data"][0]["url"])
    # store the response in the dataframe 
    df_fake.at[index, 'promt_dalle'] = PROMPT
    df_fake.at[index, 'dalle_url'] = response_url 

print("Done with image generation generation.")
print(df_fake.shape)
print(df_fake.columns)
print(df_fake['reviewId'].head(10))

In [ ]:
PROMPT = "A high quality photo of a disgusting experience at a hotel, photography, high resolution, 4k, many details"

with open("apikey_mama.txt", "r") as f:
    openai.api_key = f.read()

response = openai.Image.create(
    prompt=PROMPT,
    n=2,
    size="256x256",
)

print(response["data"][0]["url"])

with open("02_Visualizations\dalle-2-tries.txt", "a") as f:
    f.write("\nPROMPT " + PROMPT +": " + response["data"][0]["url"] + " ")

## Text (OUTDATED) ChatGPT3.5 Turbo Verworfen

In [ ]:
class chatGPT: 
    def __init__(self, api_key, rolle):
        openai.api_key = api_key
        self.dialog = [{"role":"system", "content":rolle}]

    def question(self, question):
        self.dialog.append({"role":"user", "content":question})
        ergebnis = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=self.dialog)
        antwort = ergebnis["choices"][0].message.content
        self.dialog.append({"role":"assistant", "content":antwort})
        return antwort

In [ ]:
role = "In stlye of a google user that writes a review about a place he or she visited."
prompt = "Write a Google Maps review about a bowling alley with max 200 tokens."
chatGPT(api_key, role).question(prompt)
print(chatGPT(api_key, role).question(prompt))

In [ ]:
def get_keywords(text):
    keywords = model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=5)
    keywords = [keyword[0] for keyword in keywords]
    return keywords

output_string= "I recently visited the Italian Restaurant and overall had a great experience. The atmosphere was cozy and inviting with a classic Italian vibe. The staff was friendly and attentive, making sure we had everything we needed throughout our meal.The menu had a great selection of traditional Italian dishes and some unique options as well. I tried the gnocchi and it was delicious, perfectly cooked with a flavorful sauce. My friend had the lasagna and said it was some of the best she's ever had. The only downside was the wait time for our table, as we had to wait about 30 minutes even though we had reservations. However, the food and service made up for it. Overall, I would recommend the Italian Restaurant to anyone who loves Italian cuisine and wants to have a nice, cozy dinner out with friends or family."
keywords = get_keywords(output_string)
